In [1]:
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from src.consts import *

load_dotenv(override=True)
openai = OpenAI()
anthropic = Anthropic() 

# Load the data files
oracle_df = pd.read_csv('ThePauperCube_oracle_with_pt.csv')
print(f"Loaded {len(oracle_df)} cards from oracle_df")
print(f"Columns available: {list(oracle_df.columns)}")
oracle_df.head()

Loaded 450 cards from oracle_df
Columns available: ['name', 'CMC', 'Type', 'Color', 'Color Category', 'Oracle Text', 'tags', 'MTGO ID', 'Power', 'Toughness']


,name,CMC,Type,Color,Color Category,Oracle Text,tags,MTGO ID,Power,Toughness
0,Boros Elite,1,Creature - Human Soldier,W,White,Battalion — Whenever this creature and at leas...,NaN,120547.0,1.0,1.0
1,Deftblade Elite,1,Creature - Human Soldier,W,White,"Provoke (Whenever this creature attacks, you m...",NaN,18617.0,1.0,1.0
2,Doomed Traveler,1,Creature - Human Soldier,W,White,"When this creature dies, create a 1/1 white Sp...",GTC Update;token generator;WR tokens;WG tokens,42650.0,1.0,1.0
3,Elite Vanguard,1,Creature - Human Soldier,W,White,NaN,EMA Update,60565.0,2.0,1.0
4,Faerie Guidemother,1,Creature - Faerie,W,White,Flying // Target creature gets +2/+1 and gains...,ELD Update,78110.0,1.0,1.0


# Theme Validation
Let's check if we have enough cards available for each theme in our jumpstart cube.

# Deck Construction
Now let's test the deck construction function to build actual jumpstart decks from our themes.

In [8]:
# Test the refactored deck construction function

# Build all jumpstart decks using the new refactored version
from src.construct import construct_jumpstart_decks, print_detailed_deck_analysis, CardConstraints, analyze_deck_composition

# Create constraints with custom target deck size and non-land limits
constraints = CardConstraints(
    target_deck_size=13,
    max_lands_dual=1,
    max_lands_mono=1,
    total_non_land=12,  # All 12 cards should be non-lands
)

print("🚀 Starting deck construction with refactored algorithm...")
deck_dataframes = construct_jumpstart_decks(oracle_df, constraints=constraints)

# Generate analysis first, then print detailed analysis
analysis = analyze_deck_composition(deck_dataframes)
print_detailed_deck_analysis(deck_dataframes, analysis, constraints)

🚀 Starting deck construction with refactored algorithm...
🏗️ CONSTRUCTING JUMPSTART DECKS

🔒 Phase 0: Core card reservation
Ensuring each theme gets its defining cards before general competition...

🎯 White Soldiers: Reserving core cards
  ✅ Boros Elite               |   5.3 pts | Creature - Human Sol...
  ✅ Miner's Guidewing         |   5.3 pts | Creature - Bird
  ✅ Topan Freeblade           |   4.8 pts | Creature - Human Sol...
  ✅ Ainok Bond-Kin            |   4.3 pts | Creature - Dog Soldi...
  📦 Reserved 4 core cards

🎯 White Equipment: Reserving core cards
  ✅ Mandibular Kite           |  12.8 pts | Creature Artifact - ...
  ✅ Glimmerlight              |  11.8 pts | Artifact - Equipment
  ✅ Vulshok Morningstar       |  11.8 pts | Artifact - Equipment
  ✅ Ancestral Blade           |  11.4 pts | Creature Artifact - ...
  ✅ Flayer Husk               |  11.4 pts | Creature Artifact - ...
  📦 Reserved 5 core cards

🎯 White Angels: Reserving core cards
  ✅ Combat Professor          |  

In [3]:
from src.export import export_cube_to_csv


export_cube_to_csv(deck_dataframes, 'jumpstart_decks.csv', oracle_df)

Exporting jumpstart cube to jumpstart_decks.csv...
Adding 60 unassigned cards...
✅ Successfully exported 450 cards to jumpstart_decks.csv

📊 Export Summary:
Total cards: 450
Assigned cards: 390
Unassigned cards: 60
Number of decks: 30

Deck breakdown:
  Green Elves: 13 cards
  Simic Ramp Control: 13 cards
  Boros Equipment Aggro: 13 cards
  Golgari Graveyard Value: 13 cards
  Izzet Spells Matter: 13 cards
  Orzhov Lifegain Value: 13 cards
  Selesnya Value: 13 cards
  Gruul Midrange: 13 cards
  Rakdos Aggro: 13 cards
  Dimir Mill: 13 cards
  Unassigned: 60 cards
  ... and 20 more decks


'jumpstart_decks.csv'

In [9]:
# Import validation functions and run card uniqueness validation
from src.validation import validate_card_uniqueness, validate_deck_constraints, validate_jumpstart_cube, display_validation_summary

# Run the validation
# validation_result = validate_card_uniqueness(deck_dataframes)
validate_jumpstart_cube(deck_dataframes, oracle_df, ALL_THEMES, constraints)

🎯 COMPREHENSIVE JUMPSTART CUBE VALIDATION
🔍 VALIDATING CARD UNIQUENESS
📊 VALIDATION RESULTS:
Total cards across all decks: 390
Unique cards used: 390
Duplicate cards found: 0

✅ VALIDATION PASSED!
All 390 cards are used exactly once.
🔍 VALIDATING DECK CONSTRAINTS
📊 CONSTRAINT VALIDATION RESULTS:
Valid decks: 30/30
Constraint violations: 0

✅ ALL CONSTRAINTS SATISFIED!

📈 CARD DISTRIBUTION ANALYSIS
📊 OVERALL STATISTICS:
Total cards available: 450
Total cards used: 390
Cards unused: 60
Usage rate: 86.7%

🎨 USAGE BY COLOR:
  White    :  60/ 67 cards ( 89.6%)
  Blue     :  62/ 66 cards ( 93.9%)
  Black    :  56/ 66 cards ( 84.8%)
  Red      :  63/ 68 cards ( 92.6%)
  Green    :  60/ 66 cards ( 90.9%)
  Colorless:  46/ 54 cards ( 85.2%)

🎯 DECK COMPLETENESS:
Complete decks (13 cards): 30
Incomplete decks: 0

📋 UNUSED CARDS ANALYSIS:
Unused creatures: 15
Unused lands: 23
Unused spells: 22

Sample unused cards:
  • Alabaster Host Intercessor (Creature - Phyrexian Samurai) - W
  • Raise the Al

{'overall_valid': True,
 'uniqueness': {'valid': True,
  'total_cards': 390,
  'unique_cards': 390,
  'duplicates': {},
  'duplicate_count': 0,
  'extra_instances': 0},
 'constraints': {'valid': True,
  'valid_decks': 30,
  'total_decks': 30,
  'violations': []},
 'distribution': {'total_available': 450,
  'total_used': 390,
  'unused_count': 60,
  'usage_rate': 0.8666666666666667,
  'color_stats': {'W': {'used': 60,
    'available': 67,
    'rate': 89.55223880597015},
   'U': {'used': 62, 'available': 66, 'rate': 93.93939393939394},
   'B': {'used': 56, 'available': 66, 'rate': 84.84848484848484},
   'R': {'used': 63, 'available': 68, 'rate': 92.64705882352942},
   'G': {'used': 60, 'available': 66, 'rate': 90.9090909090909},
   'C': {'used': 46, 'available': 54, 'rate': 85.18518518518519}},
  'complete_decks': 30,
  'incomplete_decks': []}}

In [5]:
# Additional analysis using the imported validation functions
from src.validation import analyze_card_distribution

# Run the distribution analysis
distribution_analysis = analyze_card_distribution(deck_dataframes, oracle_df, constraints)


📈 CARD DISTRIBUTION ANALYSIS
📊 OVERALL STATISTICS:
Total cards available: 450
Total cards used: 390
Cards unused: 60
Usage rate: 86.7%

🎨 USAGE BY COLOR:
  White    :  60/ 67 cards ( 89.6%)
  Blue     :  62/ 66 cards ( 93.9%)
  Black    :  56/ 66 cards ( 84.8%)
  Red      :  63/ 68 cards ( 92.6%)
  Green    :  60/ 66 cards ( 90.9%)
  Colorless:  46/ 54 cards ( 85.2%)

🎯 DECK COMPLETENESS:
Complete decks (13 cards): 30
Incomplete decks: 0

📋 UNUSED CARDS ANALYSIS:
Unused creatures: 15
Unused lands: 23
Unused spells: 22

Sample unused cards:
  • Alabaster Host Intercessor (Creature - Phyrexian Samurai) - W
  • Raise the Alarm (Creature Instant) - W
  • Borrowed Grace (Instant) - W
  • Prismatic Strands (Instant) - W
  • Imperial Oath (Creature Sorcery) - W
  • Triplicate Spirits (Creature Sorcery) - W
  • Oblivion Ring (Enchantment) - W
  • Bubble Snare (Enchantment - Aura) - U
  • Unable to Scream (Enchantment - Aura) - U
  • Weakstone's Subjugation (Enchantment - Aura) - U


In [6]:
# from src.generate import generate_all_theme_prompts


# all_prompts = generate_all_theme_prompts()

In [7]:
# Generate printable deck dividers for physical storage
# from src.generate import generate_all_deck_dividers, print_single_divider

# print("=== GENERATING DECK DIVIDERS ===")
# print("Creating printable divider cards for physical deck organization...")
# print()

# # Generate all dividers and save to file
# dividers = generate_all_deck_dividers(deck_dataframes, "jumpstart_deck_dividers.txt")

# print()
# print("Example divider for Selesnya Value:")
# print("-" * 60)

# Show an example divider
# if 'Selesnya Value' in deck_dataframes:
#     print_single_divider('Selesnya Value', deck_dataframes['Selesnya Value'])
# else:
#     # Show first available theme as example
#     first_theme = next(iter(deck_dataframes.keys()))
#     print(f"Showing example for {first_theme}:")
#     print_single_divider(first_theme, deck_dataframes[first_theme])

# Generated
 
Below is AI

# Theme Color Refactoring Complete ✅

Successfully refactored all hardcoded color values in the theme definitions to use the `MagicColor` enum:

## Changes Made:
- **Mono-color themes**: Updated all `['W']`, `['U']`, `['B']`, `['R']`, `['G']` to use `[MagicColor.WHITE.value]` etc.
- **Dual-color themes**: Updated all combinations like `['W', 'U']` to use `[MagicColor.WHITE.value, MagicColor.BLUE.value]`
- **COLOR_IDENTITY_MAP**: Updated to use enum values instead of hardcoded strings

## Validation Results:
✅ **All 30 decks constructed successfully**  
✅ **All card uniqueness constraints satisfied**  
✅ **All deck constraints satisfied**  
✅ **390/450 cards used (86.7% usage rate)**  

The enum refactoring maintains full backward compatibility while providing better code maintainability and consistency across the entire codebase.